In [ ]:
from utils_demo import *
from loguru import logger
from unittest.mock import patch
import warnings
warnings.filterwarnings('ignore')

: 

<img src="recommendation_system_diagram.png"
     alt="Rec System"
     style="width: 55vw; float: left; margin-right: 10px;" />

### Fetching IOI job and decomposing it

![App Decomposer Picture](app_decomposer.png)

In [ ]:
fig = plot_detected_phases(jobid=3911, merge=False, show_phases=False, width=1200, height=600)
fig.show() 

In [ ]:
%%capture
logger.remove()
def decompose_ioi_job(jobid):
    with patch.object(ComplexDecomposer, 'get_job_timeseries') as mock_get_timeseries:
        with patch.object(Configuration, 'get_kc_token') as mock_get_kc_token:
            mock_get_timeseries.return_value = get_job_timeseries_from_file(job_id=jobid)
            mock_get_kc_token.return_value = 'token'
            # init the job decomposer
            cd = ComplexDecomposer()
            return cd
        
# Launch decomposition on the signal
cd = decompose_ioi_job(jobid=3911)
compute, reads, read_bw, writes, write_bw = cd.get_job_representation()
# This is the app encoding representation for Execution Simulator
print(f"compute={compute}, reads={reads}, read_bw={read_bw}")
print(f"compute={compute}, writes={writes}, write_bw={write_bw}")
# Normalize signals to seconds and MB
timestamps = (cd.timestamps.flatten() - cd.timestamps.flatten()[0])/5
original_read =  cd.read_signal.flatten()/1e6
original_write = cd.write_signal.flatten()/1e6


Now we obtain a representation of the job that "removes" the performance of the storage tier:
- Events (timestamps) = [0, 1, 7, 11, 12, 18, 20, 23, 35, 48, 59, 65, 66]
- Reads (volumes in GB) = [0.0, 23.39, 31.69, 0.0, 28.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
- Writes (volumes in GB) = [0.0, 0.0, 0.0, 0.0, 0.0, 2.07, 9.79, 3.51, 4.32, 21.23, 10.67, 4.58, 0.0]

In [ ]:
%%capture
print(compute)
print(list(map(lambda x: round(x/1e9, 2), reads)))
print(list(map(lambda x: round(x/1e9, 2), writes)))


Signal reconstruction from the representation:
- we reuse measured bandwidths for both read and write signal
- we feed the Simulator with this data, instead of providing a Storage Performance Model: bw = f(IO_size, IO_pattern, ...)

In [ ]:
# Apply same BW as measured by IOI
read_bw = list(map(lambda x: x/1e6, read_bw))
write_bw = list(map(lambda x: x/1e6, write_bw))
io_bw = list(map(lambda x, y: x + y, read_bw, write_bw))

### Feeding App Simulator with the representation
We use the representation as an input for Execution Simulator
![App Decomposer Picture](cluster_simulator.png)

In [ ]:
sim_time, sim_read_bw, sim_write_bw = simulate_app(compute, reads, writes, io_bw, app_name="3911")


Plotting Simulation versus Origial (IOI) signals

In [ ]:
fig = display_original_sim_signals((sim_time, sim_read_bw, sim_write_bw),
                                   (timestamps, original_read, original_write),
                                   width=1400, height=900)
fig.show()

- Until now, we just replayed the representation into the simulator.
- But the aim of the Simulator is to predict the expected execution time for a different storage tier.
- The Simualtor will accept later the performance model for a tier bw = f(IO_size, IO_pattern,...)
- For now we will __mock__ the model with a hardcoded io_bandwidth

In [ ]:
# We simulate that the app data will be placed in a storage backend having the following perfs
tier_mean_read_bw = 5e3
tier_mean_write_bw = 1e3
io_bw = list(map(lambda x, y: tier_mean_read_bw if x else 0 + tier_mean_write_bw if y else 0, read_bw, write_bw))

And rerun the simulation

In [ ]:
fig = display_original_sim_signals(simulate_app(compute, reads, writes, io_bw, app_name="3911"),
                                   (timestamps, original_read, original_write),
                                   width=1400, height=900)
fig.show()

The expected duration of the application is 129x5s = 645 seconds with this storage instead of 97x5s = 485 seconds.

Conclusion and further improvements:
- fix bugs on the AppDecompser 0.1
- update simulator to accept phases having both read and write I/O
- build a Performance Model with bandwidths measured directly from the storage tier
- progressing toward simulating workflows having many jobs in parallel (already supported in App Simulator) and recommending placement for each step.